In [2]:
import json
import pandas as pd
import os
import numpy as np
import glob
import shutil

In [ ]:
def add_mean(data, filepath):
    benchmark_score = np.mean(data["test_actual"]) 
    data["avg_benchmark_score"] = benchmark_score
    print(f"MATH mean score: {benchmark_score}")

    with open(filepath, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"🗂 Updated {filepath} data\n")



In [ ]:
def load_probe_results(filepath):
    """Load probe predictions and compute detailed metrics if needed."""
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

In [ ]:
# Find all JSON files in the MATH directory
math_dir = "../../probe_results/DATA/SR_DATA/MATH"
json_files = [f for f in glob.glob(f"{math_dir}/**/*.json", recursive=True) if "k_50" in f] #specify k_50 to stop get resutls with rollouts

# Update all JSON files in the current working directory
for file_path in json_files:
    # print(file_path)
    data  = load_probe_results(file_path)
    try:
        print(data["best_layer"])
        add_mean(data, file_path)
    except:
        pass